## 基于LangChain与DeepSeek模型编写一个简单的对话Demo
DeepSeek需要安装langchain-deepseek，具体指令：
```shell
%pip install -qU langchain-deepseek
```

### 第一步：设置环境变量
DEEPSEEK_API_KEY：调用DeepSeek模型API的key，用于接口认证

In [1]:
import os
import getpass

os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("请输入DeepSeek的API Key:")

请输入DeepSeek的API Key: ········


### 第二步：初始化llm实例

In [2]:
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model="deepseek-chat", temperature=0.2, max_retries=2)

### 第三步：构建对话，并调用模型获取返回数据。对话构建方式分为多种：基础消息、消息模版

#### 第一种：使用基础消息构建对话的输入。
消息类型：
* SystemMessage：用于初始化模型行为，通常作为一系列输入消息中的第一个传入，类似于角色的设定或者对话的背景信息
* HumanMessage：代表用发送的消息，是对话的起点或者对模型的提问
* AIMessage：由AI生成的消息，用于在对话中提供信息或者响应
* FunctionMessage：代表函数调用返回的结果，包含生成的此结果所调用的函数的名称
* ToolMessage：代表工具调用的结果，包含调用生成此结果的工具的ID

消息结构：role、content和response_metadata
* role：
* content：
* response_metadata：

In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [6]:
requestMessages = [
    SystemMessage(content="你是我的旅游小助手"),
    HumanMessage(content="制作一份北京七日游攻略")
]

responseMessage = llm.invoke(requestMessages) 

responseMessage.content

'### **北京七日游攻略**  \n**主题：经典景点+文化体验+美食探索**  \n\n---\n\n#### **📅 行程概览**  \n- **Day 1**：天安门广场→故宫→景山公园→王府井  \n- **Day 2**：八达岭长城→明十三陵（定陵/长陵）→鸟巢/水立方夜景  \n- **Day 3**：颐和园→圆明园→清华/北大校门打卡  \n- **Day 4**：天坛→前门大街→大栅栏→老北京胡同游（什刹海、南锣鼓巷）  \n- **Day 5**：雍和宫→孔庙国子监→五道营胡同→簋街夜宵  \n- **Day 6**：国家博物馆→798艺术区→三里屯  \n- **Day 7**：自由活动（可选：北海公园、恭王府、京剧体验）  \n\n---\n\n#### **📍每日详细安排**  \n\n**Day 1：皇城根下的历史**  \n- **上午**：  \n  - **天安门广场**（看升旗需早起，约5:00-6:00，提前查时间）。  \n  - **故宫**（8:30开门，建议提前7天预约，中轴线+珍宝馆必看）。  \n- **下午**：  \n  - **景山公园**（登顶俯瞰故宫全景）。  \n  - **王府井**（购物+小吃，推荐“吴裕泰”花茶冰淇淋）。  \n\n**Day 2：长城豪情与奥运风采**  \n- **全天**：  \n  - **八达岭长城**（建议早6点出发，乘高铁或旅游专线车，登北八楼）。  \n  - **明十三陵**（选定陵或长陵，了解明代陵墓文化）。  \n- **晚上**：  \n  - **鸟巢+水立方**（夜景灯光超美，适合拍照）。  \n\n**Day 3：皇家园林与名校之旅**  \n- **上午**：**颐和园**（重点游长廊、昆明湖、佛香阁）。  \n- **下午**：**圆明园**（西洋楼遗址必看，建议坐电瓶车）。  \n- **傍晚**：清华/北大校门打卡（需提前预约入校）。  \n\n**Day 4：老北京风情**  \n- **上午**：**天坛**（买联票，祈年殿+回音壁）。  \n- **下午**：**前门大街**（尝“门框胡同卤煮”“爆肚冯”）→ **大栅栏**（老字号购物）。  \n- **晚上**：**什刹海**划船或酒吧听歌，逛**南锣鼓巷**（文宇奶酪店）。  \n\

#### 第二种：使用消息模版构建对话

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [12]:
promptTemplate = ChatPromptTemplate([
    ('system', '你是我的旅游小助手，我的母语是{language}'),
    ('user', '制作一份{location}七日游攻略')
])

outputParser = StrOutputParser()

chain = promptTemplate | llm | outputParser

responseContent = chain.invoke({
    "language": "中文",
    "location": "上海"
})

responseContent

'以下是一份详细的上海七日游攻略，涵盖经典景点、文化体验、美食和周边水乡，兼顾深度与趣味性：\n\n---\n\n### **Day 1：经典地标打卡**\n- **上午**  \n  - **外滩**：欣赏万国建筑博览群，建议清晨人少时拍照。  \n  - **南京东路步行街**：步行至人民广场，感受上海商业繁华。  \n- **下午**  \n  - **豫园**（门票40元）：明代江南园林，逛城隍庙小吃街（推荐南翔小笼包）。  \n  - **BFC外滩金融中心**：现代艺术展览或露台酒吧看夜景。  \n- **晚上**  \n  - **黄浦江游船**（夜航约120元）：灯光秀下的陆家嘴全景。\n\n---\n\n### **Day 2：浦东现代之旅**\n- **上午**  \n  - **上海中心大厦**（118层观光厅180元）：俯瞰城市全景。  \n  - **陆家嘴三件套**：打卡环球金融中心、金茂大厦外观。  \n- **下午**  \n  - **浦东美术馆**（门票100元）：滨江观景台+艺术展。  \n  - **滨江大道骑行**：沿江骑行至船厂1862（工业风改造艺术区）。  \n- **晚上**  \n  - **朵云书院**（需预约）：52层免费观景，点杯咖啡看日落。\n\n---\n\n### **Day 3：文艺与历史**\n- **上午**  \n  - **武康路**：骑行梧桐区，打卡武康大楼、巴金故居。  \n  - **安福路**：逛多抓鱼书店、小众咖啡馆。  \n- **下午**  \n  - **上海博物馆**（免费）：青铜器/书画精品（提前预约）。  \n  - **思南公馆**：老洋房群，周末有市集。  \n- **晚上**  \n  - **田子坊**（商业化但夜景不错）+ 泰康路鲜肉月饼。\n\n---\n\n### **Day 4：亲子/乐园日**\n- **选项1**：迪士尼乐园（必玩创极速光轮、加勒比海盗）。  \n- **选项2**：海昌海洋公园（虎鲸表演、火山漂流）。  \n- **Tips**：迪士尼建议早享卡，晚上看烟花秀。\n\n---\n\n### **Day 5：周边水乡一日游**\n- **朱家角**（地铁17号线直达）：  \n  - 游放生桥、扎肉粽，坐乌篷船（60元/船）。  \n-